In [1]:
#Reset testbed
def reset_testbed() :
    #!cd ..; cd..; sudo ./setup_testbed.sh; sudo ip netns exec ns_mid ./../../setup_ns_mid.sh;
    print("testbed is empty")
    

In [2]:
from shutil import copyfile
import os.path

def save_measurement(owd, container, duration, counter) :
    
    save_directory    = "../saves/"+container+"/csv/"
    owd_str           = "owd" + str(owd)
    veths             = ["veth0_stats","veth3_stats","veth0_aggr","veth3_aggr"]
    peers             = ['peer1','peer2','peer3','peer4','peer5']
    duration_str      = str(duration * 25)
    
    for veth in veths : 
        src_file = veth + ".csv"
        dst_file = veth + "_" + owd_str + "_" + str(counter) + ".csv"
        src_path = "../temp/" + src_file
        dst_path = save_directory + dst_file
        copyfile(src_path, dst_path)
        
    for peer in peers : 
        src_file = "owd_1.csv"
        dst_file = peer + "_owd_1_" + owd_str + "_" + str(counter) + ".csv"
        src_path = "../temp/" + peer + "/" + src_file
        dst_path = save_directory + dst_file
        if(os.path.isfile(src_path)) :
            copyfile(src_path, dst_path)
        
        src_file = "fecstat.csv"
        dst_file = peer + "_fecstat_" + owd_str + "_" + str(counter) + ".csv"
        src_path = "../temp/" + peer + "/" + src_file
        dst_path = save_directory + dst_file
        if(os.path.isfile(src_path)) :
            copyfile(src_path, dst_path)

        
    #PSNR metric
    #!cd ..; cd ..; vqmt source.yuv destination.yuv 352 288 $duration_str 1 results PSNR SSIM MSSSIM VIFP
    dst_path = save_directory + "psnr_result_" + owd_str + "_" + str(counter) + ".csv"
    src_path="../../results_psnr.csv"
    if(os.path.isfile(src_path)) :
        copyfile(src_path, dst_path)
    
    dst_path = save_directory + "ssim_result_" + owd_str + "_" + str(counter) + ".csv"
    src_path="../../results_ssim.csv"
    if(os.path.isfile(src_path)) :
        copyfile(src_path, dst_path)
    
    dst_path = save_directory + "msssim_result_" + owd_str + "_" + str(counter) + ".csv"
    src_path="../../results_msssim.csv"
    if(os.path.isfile(src_path)) :
        copyfile(src_path, dst_path)
    
    dst_path = save_directory + "vifp_result_" + owd_str + "_" + str(counter) + ".csv"
    src_path="../../results_vifp.csv"
    if(os.path.isfile(src_path)) :
        copyfile(src_path, dst_path)


In [3]:
from shutil import copyfile
import os.path
import pandas as pd

#range_ = 1200
#owd = 100
#container = "rmcat1"
def eval_measurements(owd, container, time_range_):
    save_directory    = "../saves/"+container+"/csv/"
    owd_str           = "owd" + str(owd)
    columns = ['sender_rate','fec_rate','capacity']
    alldf = pd.DataFrame(columns=columns)
    alldf = alldf.fillna(0) # with 0s rather than NaNs
    for x in range(0, 9):
        csv_file = save_directory + "veth_stats_" + owd_str + "_" + str(x) + ".csv"
        if(os.path.isfile(csv_file) == False) :
            print('Missing file: {}'.format(csv_file))
            continue

        df = pd.read_csv(csv_file, names=columns)
        alldf = pd.concat([alldf, df[:time_range_]])
    
    print("\nAverages\n" + str(alldf.mean()))
    print("\nStandard deviations\n" + str(alldf.std()))



In [4]:
#Plotting 
def plot_rmcat1(owd, container, veth_str,labels, legends, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    path_delay_str  = str(owd * 1000)

    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    fecstat_file   = "../saves/" + container + "/csv/" + peer_str + "_fecstat_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat1.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_owd_1_result" + str(owd) + "_" + pdf_num_str + ".pdf"
    
    !gnuplot -e "fecstat_file='$fecstat_file'" -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "path_delay='$path_delay_str'" $plot_file        
    
#Plotting 
def plot_rmcat2(owd, container, veth_str, labels, legends, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    peer2_str       = "peer2"
    labels_str      = str(labels)
    legends_str     = str(legends)
    path_delay_str  = str(owd * 1000)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str  + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file2  = "../saves/" + container + "/csv/" + peer2_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat2.plot"
    fecstat_file    = "../saves/" + container + "/csv/" + peer_str + "_fecstat_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    fecstat_file2   = "../saves/" + container + "/csv/" + peer2_str + "_fecstat_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_result" + str(owd) + "_" + pdf_num_str + ".pdf"
    
    !gnuplot -e "fecstat_file='$fecstat_file'" -e "fecstat_file2='$fecstat_file2'" -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "path_delay='$path_delay_str'" -e "owd_file='$owd_file'" -e "owd_file2='$owd_file2'" $plot_file
#-e "labels='" + str(labels) + "'" -e "legends='" + str(legends) + "'"

def plot_rmcat3(owd, container, veth_str, veth_str2, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    peer2_str       = "peer2"
    labels_str      = str(labels)
    legends_str     = str(legends)
    path_delay_str  = str(owd * 1000)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    throughput_file2 = "../saves/" + container + "/csv/" + veth_str2 + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str  + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file2  = "../saves/" + container + "/csv/" + peer2_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat3.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_result" + str(owd) + "_" + pdf_num_str + ".pdf"
    
    !gnuplot -e "throughput_file='$throughput_file'" -e "throughput_file2='$throughput_file2'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "path_delay='$path_delay_str'" -e "owd_file2='$owd_file2'" $plot_file

def plot_rmcat4(owd, container, veth_str, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    peer2_str       = "peer2"
    peer3_str       = "peer3"    
    labels_str      = str(labels)
    legends_str     = str(legends)
    path_delay_str  = str(owd * 1000)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str  + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file2  = "../saves/" + container + "/csv/" + peer2_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file3  = "../saves/" + container + "/csv/" + peer3_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat4.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_result" + str(owd) + "_" + pdf_num_str + ".pdf"
    
    !gnuplot -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "owd_file='$owd_file'" -e "path_delay='$path_delay_str'" -e "owd_file2='$owd_file2'" -e "owd_file3='$owd_file3'" $plot_file

#Plotting 
def plot_rmcat5(owd, container, veth_str, labels, legends, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    peer2_str       = "peer2"
    peer3_str       = "peer3"
    peer4_str       = "peer4"
    peer5_str       = "peer5"
    labels_str      = str(labels)
    legends_str     = str(legends)
    path_delay_str  = str(owd * 1000)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str  + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file2  = "../saves/" + container + "/csv/" + peer2_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file3  = "../saves/" + container + "/csv/" + peer3_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file4  = "../saves/" + container + "/csv/" + peer4_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file5  = "../saves/" + container + "/csv/" + peer5_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat5.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_result" + str(owd) + "_" + pdf_num_str + ".pdf"
    
    !gnuplot -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "path_delay='$path_delay_str'" -e "owd_file='$owd_file'" -e "owd_file2='$owd_file2'" -e "owd_file3='$owd_file3'"  -e "owd_file4='$owd_file4'"  -e "owd_file5='$owd_file5'" $plot_file
#-e "labels='" + str(labels) + "'" -e "legends='" + str(legends) + "'"


#Plotting 
def plot_rmcat6(owd, container, veth_str, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    labels_str      = str(labels)
    legends_str     = str(legends)
    path_delay_str  = str(owd * 1000)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str  + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat6.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_result" + str(owd) + "_" + pdf_num_str + ".pdf"
    
    !gnuplot -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "path_delay='$path_delay_str'" -e "owd_file='$owd_file'" $plot_file
#-e "labels='" + str(labels) + "'" -e "legends='" + str(legends) + "'"

#Plotting 
def plot_rmcat7(owd, container, veth_str, pdf_num) :
    
    pdf_num_str     = str(pdf_num)
    peer_str        = "peer1"
    peer2_str       = "peer2"
    labels_str      = str(labels)
    legends_str     = str(legends)
    path_delay_str  = str(owd * 1000)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file   = "../saves/" + container + "/csv/" + peer_str  + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    owd_file2  = "../saves/" + container + "/csv/" + peer2_str + "_owd_1_owd" + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file  = "../gnuplots/rmcat7.plot"
    output     = "../saves/" + container + "/pdfs/" + peer_str + "_result" + str(owd) + "_" + pdf_num_str + ".pdf"
    
    !gnuplot -e "throughput_file='$throughput_file'" -e "output_file='$output'" -e "path_delay='$path_delay_str'" -e "owd_file='$owd_file'" -e "owd_file2='$owd_file2'" $plot_file


In [5]:
save_measurement(50, "rmcat1", 100, 0)
plot_rmcat1(50, "rmcat1", "veth0_stats", 0, 0, 0)

save_measurement(10, "rmcat7", 300, 0)
plot_rmcat7(10, "rmcat7", "veth0_stats", 0)

save_measurement(300, "rmcat2", 120, 0)
plot_rmcat2(300, "rmcat2", "veth0_stats", 0, 0, 0)

#plot_rmcat2(50, "rmcat2", "veth0_stats", 0, 0, 0)
#plot_rmcat3(50, "rmcat3", "veth0_stats","veth3_stats", 0)
#plot_rmcat4(50, "rmcat4", "veth0_stats", 0)
#plot_rmcat5(50, "rmcat5", "veth0_stats", 0, 0, 0)
#plot_rmcat6(50, "rmcat6", "veth0_stats", 0)
#plot_rmcat7(50, "rmcat7", "veth0_stats", 0)

NameError: name 'labels' is not defined

In [6]:
#RMCAT1 Test

container = "rmcat1"
veth0_str = "veth0_stats"
duration  = 100
range_    = 3000
flownum   = 1

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
#labels    = [True,False,False]
labels    = [False,False,False]
#legends   = [False,False,True]
legends   = [False,False,True]

#Reset testbed
reset_testbed();


for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
            
        !cd ..;cd ..; echo "" > log_rmcat1_test.txt
        !cd ..;cd ..; scripts/runs/rmcat1.sh -s $owd -r 10 >> log_rmcat1_test.txt
        save_measurement(owd, container, duration, y)
        plot_rmcat1(owd, container, veth0_str, label, legend, y)
        print("test for owd: " + str(owd) + " is done, counter: " + str(y))
    
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
         
    eval_measurements(owd, container, duration * 10)
    
    

testbed is empty
rm: cannot remove 'scripts/temp/peer3/*': No such file or directory

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
test for owd: 50 is done, counter: 0
rm: cannot remove 'scripts/temp/peer1/*': No such file or directory
rm: cannot remove 'scripts/temp/peer2/*': No such file or directory
rm: cannot remove 'scripts/temp/peer3/*': No such file or directory
*** Error in `ntrt': double free or corruption (out): 0x00000000023507c0 ***
======= Backtrace: =========
/lib/x86_64-linux-gnu/libc.so.6(+0x77725)[0x7f896f1f0725]
/lib/x86_64-linux-gnu/libc.so.6(+0x7ff4a)[0x7f896f1f8f4

In [683]:
#RMCAT2 Test

container = "rmcat2"
veth0_str = "veth0_stats"
duration  = 120
range_    = 3000
flownum   = 2

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
#labels    = [True,False,False]
#legends   = [False,False,True]
labels    = [False,False,False]
legends   = [False,False,False]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_rmcat2_test.txt

for x in range(0,3) :

    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
        !cd ..;cd ..; scripts2/runs/rmcat2.sh -s $owd >> log_rmcat2_test.txt
        save_measurement(owd, container, duration, y)
        plot_rmcat2(owd, container, veth0_str, label, legend, y)
        #plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
    
    
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    

testbed is empty
rm: cannot remove ‘scripts2/temp/peer2/*’: No such file or directory
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice

In [684]:
#RMCAT3 Test

container = "rmcat3"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 100
range_    = 3000
flownum   = 1

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 1, 1, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_rmcat3_test.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
        !cd ..;cd ..; scripts2/runs/rmcat3.sh -s $owd -r $owd >> log_rmcat3_test.txt
        save_measurement(owd, container, duration,  y)
        plot_rmcat3(owd, container, veth0_str, veth3_str, y)
        #plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
        #plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth3_str)
    
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    
    

testbed is empty
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  ou

In [685]:
#RMCAT4 Test

container = "rmcat4"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 120
range_    = 4000
flownum   = 3

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_test_rmcat4.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
            
        !cd ..;cd ..; scripts2/runs/rmcat4.sh -s $owd >> log_test_rmcat4.txt
        save_measurement(owd, container, duration,  y)
        plot_rmcat4(owd, container, veth0_str, y)
        
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    
    

testbed is empty
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).

Usa

In [686]:
#RMCAT5 Test

container = "rmcat5"
veth0_str = "veth0_stats"
duration  = 120
range_    = 3000
flownum   = 2

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
#labels    = [True,False,False]
#legends   = [False,False,True]
labels    = [False,False,False]
legends   = [False,False,False]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_rmcat5_test.txt

for x in range(0,3) :

    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
        !cd ..;cd ..; scripts2/runs/rmcat5.sh -s 10 -r 50 >> log_rmcat5_test.txt
        save_measurement(owd, container, duration, y)
        plot_rmcat5(owd, container, veth0_str, label, legend, y)
        #plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
    
    
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    

testbed is empty

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            sen

In [ ]:
#RMCAT6 Test

container = "rmcat6"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 300
range_    = 2000
flownum   = 1

plays     = [1,0,0] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_test_rmcat6.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
            
        !cd ..;cd ..; scripts/runs/rmcat6.sh -s $owd >> log_test_rmcat6.txt
        save_measurement(owd, container, duration,  y)
        plot_rmcat6(owd, container, veth0_str, y)
        print("test for owd: " + str(owd) + " is done, counter: " + str(y))
        
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    
    

In [688]:
#RMCAT7 Test

container = "rmcat7"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 300
range_    = 2000
flownum   = 1

plays     = [1,1,1] 
evals     = [0,0,0]

#             0  1  2  3  4  5  6  7  8  9
plays_mc    = [
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,],
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]
              [1, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
              ]

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

#Reset testbed
reset_testbed();

!cd ..;cd ..; echo "" > log_test_rmcat7.txt

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(0, 10):
            
        if(plays_mc[x][y] == 0) :
            continue;
            
        !cd ..;cd ..; scripts2/runs/rmcat7.sh -s $owd >> log_test_rmcat7.txt
        save_measurement(owd, container, duration,  y)
        plot_rmcat7(owd, container, veth0_str, y)
        
for x in range(0,3) :
    
    if(evals[x] == 0) :
        continue;
    
    eval_measurements(owd, container, duration * 10)
    
    

testbed is empty
rm: cannot remove ‘scripts2/temp/peer2/*’: No such file or directory
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).
rm: cannot remove ‘scripts2/temp/peer3/*’: No such file or directory

Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice

In [689]:
!pm-suspend

In [ ]:
import numpy
import sys

#output = sys.argv[1] # prints var1
output = "output.dat"

for x in range(0,10) :
    idle = numpy.random.exponential(10)
    !sleep $idle
    size = str(int(numpy.random.uniform(100, 10240)))
    !dd if=/dev/zero of=$output  bs="$size"KB  count=1 > log.txt
    !iperf -c 127.0.0.1 -F $output -p 1234 > log.txt

